## Set up Packages

In [56]:
import numpy as np
import pandas as pd
from transformers import BertTokenizer, Trainer, BertForSequenceClassification, TrainingArguments, logging,pipeline
from datasets import Dataset, load_dataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Load Model and Tokenizer

In [63]:
model = BertForSequenceClassification.from_pretrained("yiyanghkust/finbert-tone",num_labels=3)
tokenizer = BertTokenizer.from_pretrained("yiyanghkust/finbert-tone")

## Load up Dataset

In [64]:
dataset = load_dataset("financial_phrasebank", "sentences_allagree")["train"]
print(dataset)

x = dataset["sentence"]
y = dataset["label"]

x_train, x_temp, y_train,y_temp = train_test_split(x,y,test_size=0.3, random_state=42, shuffle=True)
x_test, x_eval, y_test, y_eval = train_test_split(x_temp, y_temp,test_size = 0.5, random_state = 42, shuffle = True)

train_df = pd.DataFrame({
    "sentence": x_train,
    "label": y_train
})

test_df = pd.DataFrame({
    "sentence": x_test,
    "label": y_test
})

eval_df = pd.DataFrame({
    "sentence": x_eval,
    "label": x_eval
})



train_ds = Dataset.from_pandas(train_df)
test_ds = Dataset.from_pandas(test_df)
eval_ds = Dataset.from_pandas(eval_df)

train_ds = train_ds.map(lambda x: tokenizer(x["sentence"],truncation = True, padding = "max_length", max_length = 256), batched = True)
eval_ds = eval_ds.map(lambda x: tokenizer(x["sentence"],truncation = True, padding = "max_length", max_length = 256), batched = True)
test_ds = test_ds.map(lambda x: tokenizer(x["sentence"],truncation = True, padding = "max_length", max_length = 256), batched = True)
print(eval_ds)

Dataset({
    features: ['sentence', 'label'],
    num_rows: 2264
})


Map:   0%|          | 0/1584 [00:00<?, ? examples/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Map:   0%|          | 0/340 [00:00<?, ? examples/s]

Dataset({
    features: ['sentence', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 340
})


## Evaluation Function and Argument Set Up

In [65]:
def evaluation(datapoint):
    predictions, labels = datapoint
    predictions = np.argmax(predictions,axis = 1)
    return {'accuracy': accuracy_score(predictions, labels)}

args = TrainingArguments(
    logging_dir='./logs',
    output_dir = 'temp/',
    logging_strategy='epoch',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)
trainer = Trainer(
    model=model,                         
    args=args,                  
    train_dataset=train_ds,        
    eval_dataset=eval_ds,            
    compute_metrics=evaluation
)

## Test the model before training

In [66]:
model.eval()
trainer.predict(train_ds).metrics

{'test_loss': 12.992094039916992,
 'test_accuracy': 0.029671717171717172,
 'test_runtime': 6.4857,
 'test_samples_per_second': 244.23,
 'test_steps_per_second': 7.709}

## Model Training and saving

In [7]:
trainer.train()
trainer.save_model('finbertNLP/')

wandb: Currently logged in as: kensh1nhualuoweiyang (kensh1nhualuoweiyan). Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,1.576500,0.358996,0.839646
2,0.304700,0.109633,0.965278
3,0.093500,0.021711,0.993056
4,0.032600,0.007118,0.998737
5,0.016500,0.004502,1.000000


## Test the model of test data

In [42]:
model = BertForSequenceClassification.from_pretrained('finbertNLP',num_labels=3)
tokenizer = BertTokenizer.from_pretrained('yiyanghkust/finbert-tone')
pipe = pipeline("sentiment-analysis", model=model, tokenizer=tokenizer)


def evaluation(datapoint):
    predictions, labels = datapoint
    predictions = np.argmax(predictions,axis = 1)
    return {'accuracy': accuracy_score(predictions, labels)}

args = TrainingArguments(
    logging_dir='./logs',
    output_dir = 'temp/',
    logging_strategy='epoch',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    num_train_epochs=5,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model='accuracy',
)
trainer = Trainer(
    model=model,                         
    args=args,                  
    train_dataset=train_ds,        
    eval_dataset=eval_ds,            
    compute_metrics=evaluation
)

model.eval()
trainer.predict(test_ds).metrics
# outputs = []
# outputMap = {"Neutral":1,"Positive":2,"Negative":0}

# for x in test_ds:
#     result =pipe(x["sentence"])[0]['label']
#     counter +=1
#     outputs.append(outputMap[result])

# print(f"Accuracy Score On Test Data: {accuracy_score(y_test,outputs)}")

ValueError: You have to specify either input_ids or inputs_embeds

## Link to WandB Report Board
https://wandb.ai/kensh1nhualuoweiyan/huggingface/reports/FinBERT-Fine-Tunning--Vmlldzo3MzU0OTE4